# folium 시각화
참고할 만한 링크

- folium 공식 documentation(eng) : https://python-visualization.github.io/folium/
- folium 패키지 살펴보기(kor) :  https://dailyheumsi.tistory.com/85
- choropleth(eng) :    https://vverde.github.io/blob/interactivechoropleth.html

### 패키지/모듈 임포트

In [1]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
from folium.plugins import Draw
import branca.colormap as cm
import json
import matplotlib.pyplot as plt

pd.options.display.max_rows = 150
pd.options.display.max_columns = 350
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

path1 = './data/'
path2 = './map/' 

### 파일 임포트

1) 'map'이라는 하위 폴더 만들어 아래 링크에 있는 파일 5개 넣어두기

http://bit.ly/2O83K2F
- 행정구역코드.csv
- sigungu.shp
- sido.shp
- east.png
- data.geojson

2) 'data' 폴더로 아래 두 파일 다운로드 받기

http://bit.ly/2rcyg2R
- 대구서구음주주소.csv
- 대구서구제조업체.csv

In [2]:
sido_data=pd.read_csv(path1+'imp_시도별지역안전지표_2015.csv', encoding='cp949', engine='python')
sigungu_data=pd.read_csv(path1+'imp_시군구별지역안전지표_2015.csv', encoding='cp949', engine='python')
jejo=pd.read_csv(path1+'대구서구제조업체.csv', encoding='utf-8', engine='python')
accident=pd.read_csv(path1+'대구서구음주주소.csv', encoding='cp949', engine='python')
jejo.dropna(inplace=True)
# accident.columns=['x', 'y']

#map폴더 내에 있어야 하는 파일
code=pd.read_csv(path2+'행정구역코드.csv', encoding='cp949', engine='python')
sido_shp=json.load(open(path2+'sido.json', encoding='utf=8'))
sigungu_shp=json.load(open(path2+'sigungu.json', encoding='utf=8'))

In [5]:
from pyproj import Proj, transform
import numpy as np
import pandas  as pd
import matplotlib.pyplot as plt
%matplotlib inline
path = './data/'
pd.options.display.max_rows = 150
pd.options.display.max_columns = 350
plt.rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

# Projection 정의
# UTM-K
proj_UTMK = Proj('+proj=tmerc +lat_0=38 +lon_0=127.5 +k=0.9996 +x_0=1000000 +y_0=2000000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
proj_WGS84 = Proj('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')

# Wgs84 경도/위도, GPS사용 전지구 좌표

In [6]:
# x, y 컬럼을 이용하여 UTM-K좌표를 WGS84로 변환한 Series데이터 반환
def transform_utmk_to_w84(df):
    return pd.Series(transform(proj_UTMK, proj_WGS84, df['x'], df['y']), index=['x', 'y'])

accident=pd.read_csv(path+'대구서구음주주소.csv', encoding='cp949', engine='python')
accident.columns = ['id', 'x', 'y']

accident[['x', 'y']] = accident.apply(transform_utmk_to_w84, axis=1)

In [9]:
jejo.drop(['회사명', '단지명', '설립구분', '공장대표주소', 
           '용도지역', '지목', '대기오염', '수질오염', '소음/진동여부'], axis=1, inplace=True)
jejo.columns=['y', 'x']
jejo.head(1)

,y,x
1,35.871685,128.537256


In [11]:
#시군구별 대표 위도경도값 얻기 위해 sigungu_data와 code병합
sigungu_data=pd.merge(sigungu_data, code, how='left', on=['시도','시군구'])
sigungu_data.head(1)

,년도,시도,시군구,지역,1인가구수,5대범죄 발생건수,가스사고발생건수,가해(타살) 사망자수,감염병 발생건수,감염병 사망자수,건강보험급여실적,건설업 업체수,건설업 종사자수,건축용지 면적,경찰관서수,고령인구수,공무원수(정원),공업지역 면적,교통문화지수,교통사고 발생건수,교통사고 부상자수,교통사고 사망자수,교통사고사망자수(고속도로),구거면적,구거면적 비율,구급발생건수,구조구급 발생건수,구조구급대원수,구조발생건수,급경사지 붕괴위험지구 수,기초수급자수,기초수급자수(65세이상),노인 교통사고 부상자수,노인 교통사고 사망자수,도로면적,도로면적 비율,도로연장,도시지역 면적,도시지역 면적 비율,독거노인수,무면허교통사고 발생건수,반지하가구 수,범죄발생건수(강간.강제추행),범죄발생건수(강도),범죄발생건수(도박),범죄발생건수(마약),범죄발생건수(방화),범죄발생건수(살인),범죄발생건수(약취.유인),범죄발생건수(절도),범죄발생건수(폭력),병상수,보건업 및 사회복지서비스업 종사자수,보행사상자수,보행자전용·우선도로 면적,보행자전용·우선도로 연장,비닐하우스 면적,빈집수,산사태위험지역 면적,상업지역 면적,수계밀도,순이동자수,스트레스인지율,시가화율,시군구내 전입자수,시군구외 전입자수,시도간 전입자수,시도내-시군구간 전입자수,실업률,야간인구수,양호한 주관적 건강수준 인지율,어린이 교통사고 부상자수,어린이 교통사고 사망자수,어린이 아토피,어린이 천식,외국인수,우울감경험률,운전시 안전벨트 착용률,월간음주율,유지면적,유지면적 비율,유치원생수,음식점 및 주점업 업체수,음식점 및 주점업 종사자수,음주교통사고 발생건수,의료기관수(요양기관),의료보장 공.교 가입자수,의료보장 근로자 사업장수,의료보장 사업장수,의료보험료,의료인력,이혼건수,익사자수,인구밀도,인플루엔자 예방접종률,임야 면적,자동차등록대수,자살 사망자수,자연재해 발생건수(10년평균),자연재해 사망자수,자연재해 사망자수(10년평균),자연재해 피해액(10년평균),자연재해 피해자수(10년평균),자연재해위험개선지구 수,장애인수,재난약자수,재정자립도,재정자주도,전기화재발생건수,제방면적,제방면적 비율,제조업 업체수,제조업 종사자수,주간인구지수,주거지역 면적,주민등록인구,주민등록인구(14세이하),주민등록인구(60세이상),주민등록인구(여자),주민등록인구(청소년),지역안전도(관리능력),지역안전도(등급),지역안전도(방재성능),지역안전도(위험환경),지역안전도(점수),질병이환 및 사망외인으로 인한 사망자수,창고 및 운송관련 서비스업 업체수,초등학생수,총 사업체수,총전입자수,추락 사망자수,특수의료장비수,하천면적,하천면적 비율,하천연장,학교수(고등학교),학교수(중학교),학교수(초등학교),해안선 길이,행정구역 면적,혼인귀화자수,화재 발생건수,화재 사망자수,화재 피해액,화재 피해자수,화재구조실적,화재사망자(환산),화재피해 경감액,화재,교통,자연재해,범죄,안전사고,자살,감염병,행정구역코드,Latitude,Longitude
0,2015,강원도,강릉시,2015_강원도_강릉시,1161.45445,121.96083,0.04634,0.13901,25.67109,2.03886,1559476.199,37.53354,335.85565,0.02344,0.3707,1668.57424,58.84888,0.00457,77.36,228.30585,392.99003,0.83408,0.04634,0.00641,0.64111,32.34371,44.85489,2.22421,12.51118,0.41704,314.30862,108.15219,36.42143,0.32436,0.02316,2.31551,660.14293,0.13173,13.17299,404.29643,53.98342,22.52012,4.63377,0.50971,1.29746,0.55605,0.50971,0.13901,0.0,48.65458,68.02374,108.61557,309.39682,19.69352,0.0,0.0,0.0817,229.04725,99.99013,0.00176,4.9,-37.53354,23.2,2.25,773.56156,598.26604,406.61332,191.65273,2.0,9931.4202,47.9,27.61727,0.0,98.65296,89.98781,76.78157,4.7,73.9,59.8,0.00471,0.47098,70.20162,201.29097,489.5578,53.98342,14.78173,458.8359,222.69899,224.83052,6229719.75,0.41704,21.91773,0.09268,209.07,32.2,0.80524,4428.91102,3.66068,0.00231,0.0,0.0,5886.61005,0.0,0.04634,605.49472,2261.65046,17.39,64.61,2.27055,0.00123,0.12303,53.56638,300.31463,99.4,0.01616,215807.0,1283.55429,2266.60859,5034.96179,1200.47079,0.741,4.0,0.438,0.553,59.98,7.08967,4.68011,522.87461,2564.56,1371.82761,0.78774,1.52914,0.02068,2.06752,0.02235,0.50971,0.55605,1.62182,0.07086,1040.37773,9.91627,10.79668,0.09268,72985.02829,0.88042,1.57548,0.17458,271425.0696,4,3,4,4,4,4,4,42150,37.751853,128.876057


In [5]:
#sigungu_data.loc[[""]]

---------------------
### 기본 지도 생성해보기

**tile종류**
- 기본 : 
- 밝고 깔끔 : cartodbpositron
- 어둡고 깔끔 : cartodbdark_matter

In [10]:
#기본지도생성
m = folium.Map(location=[36, 127],tiles=None, min_zoom=4, zoom_start=6.2)

#지도타일 추가
folium.TileLayer('cartodbpositron',control=True, min_zoom=5, zoom_start=6.2).add_to(m)

#동해 패치
icon = folium.features.CustomIcon(path2+'east.png',icon_size=(120,90))
folium.Marker(location=[40.2747,134.85077],icon=icon).add_to(m)

m

--------------------------
### 교통사망사고 데이터 좌표찍기

**1. 1번셀 실행하여 polygon만들기**
    1. 1번셀 실행하여 만들어진 html파일 열기
    2. polygon그리고 싶은 지역으로 지도 확대
    2. 왼쪽 선택툴 중 'Draw a polygon'기능 이용해 공단지역 표시
    3. 다 그리면 우측 'Export'를 클릭하여 geojson파일 받기
    
    
**2. 2번셀 실행하여 polygon적용된 지도 저장**

In [66]:
#1번셀

#기본지도생성
m = folium.Map(location=[36, 127], min_zoom=7, tiles=None, zoom_start=7.2)
folium.TileLayer('cartodbpositron',control=True, min_zoom=7, zoom_start=7.2).add_to(m)
icon = folium.features.CustomIcon(path2+'east.png',icon_size=(120,90))
folium.Marker(location=[40.2747,134.85077],icon=icon).add_to(m)

#마커찍을 위치정보
data=np.array([jejo.y.values, jejo.x.values]).T
data2=np.array([accident.y.values, accident.x.values]).T

# 마커찍기
# for i in data :
#     folium.Circle(location=i, radius=5,
#                   color='#8B00FF', line_weight=0,
#                   opacity=0.8).add_to(m)
for i in data2 :
    folium.Circle(location=i, radius=50,
                  color='#FFD400', line_weight=0,
                  fill_color='#FFD400',fill_opacity=1,
                  opacity=1).add_to(m)

shp=json.load(open(path2+'data.geojson', encoding='utf=8'))
folium.Choropleth(
    geo_data=shp,
    fill_color='#8B00FF',
    fill_opacity=0.3,
    line_opacity=0,
    overlay=True,
    reset=True
).add_to(m)
    
#tooltip만들기(마우스 올리면 위치에 따른 정보 보이는 것)
style_function = lambda x: {'fillColor': '#ffffff',
                            'color':'#000000',
                            'fillOpacity': 0, 
                            'weight': 0.1}

ttip = folium.features.GeoJson(
    sigungu_shp,
    style_function=style_function, 
    control=False, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['SIG_KOR_NM'], #시도(CTP_KOR_NM),시군구(SIG_KOR_NM)
        aliases=['시군구명'] #시도(시도명),시군구(시군구명)
    )
)

ttip.add_to(m)

m.keep_in_front(ttip)

draw = Draw(export=True)
draw.add_to(m)

m.save(path2+'S_Marker.html')

In [25]:
#2번셀
shp=json.load(open(path2+'data.geojson', encoding='utf=8')) #만들어진 geojson파일 넣기

#기본지도생성
m = folium.Map(location=[36, 127],tiles=None, min_zoom=7, zoom_start=7.2)
folium.TileLayer('cartodbpositron',control=True, min_zoom=7, zoom_start=7.2).add_to(m)
icon = folium.features.CustomIcon(path2+'east.png',icon_size=(120,90))
folium.Marker(location=[40.2747,134.85077],icon=icon).add_to(m)

#교통사고 사망 데이터 위치정보
data=np.array([accident.위도.values, accident.경도.values]).T

#공단지역 polygon('darw기능을 통해 저장')
folium.Choropleth(
    geo_data=shp,
    fill_color='#FFD400',
    fill_opacity=0.8,
    line_opacity=0,
    overlay=True,
    bins=bins,
    reset=True
).add_to(m)

for i in data :
    folium.Circle(location=i, radius=5, 
                  #fillcolor='#3186cc',
                  color='#8B00FF', line_weight=0,
                  opacity=0.8).add_to(m)
    
#tooltip만들기(마우스 올리면 위치에 따른 정보 보이는 것)
style_function = lambda x: {'fillColor': '#ffffff',
                            'color':'#000000',
                            'fillOpacity': 0, 
                            'weight': 0.1}

ttip = folium.features.GeoJson(
    sigungu_shp,
    style_function=style_function, 
    control=False, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['SIG_KOR_NM'], #시도(CTP_KOR_NM),시군구(SIG_KOR_NM)
        aliases=['시군구명'] #시도(시도명),시군구(시군구명)
    )
)

ttip.add_to(m)

m.keep_in_front(ttip)

folium.LayerControl(collapsed=True).add_to(m)

draw = Draw(export=True)
draw.add_to(m)

m.save('D:/S_Marker.html')

------------------